### This notebook's purpose is to get the CALCE data in a workable format (in a dataframe, cleaned, battery type labeled etc.)

### Goals: (Add to this)
1. Get files from CALCE website into dataframe(s)
2. Add label for every cycle which battery this data came from 
3. Find/Remove empty cells  (delete the whole row or should we do interpolation?) (Might not even have empty cells)


All CS2 cells underwent the same charging profile which was a standard constant current/constant voltage protocol with a constant current rate of 0.5C until the voltage reached 4.2V and then 4.2V was sustained until the charging current dropped to below 0.05A. Unless specified, the discharge cut off voltage for these batteries was 2.7V. All the CS2 cells were randomly numbered and named accordingly. Name 'CS2_n' was given for the nth numbered CS2 cell. 

#### Imports:

In [47]:
import scipy.io
import scipy.signal
import os 
import pandas as pd
from pandas import ExcelWriter
#also had to pip install openpyxl
import zipfile
import requests
import xlrd
import matplotlib
import matplotlib.pyplot as plt
import numpy as np 
import seaborn as sns
import statistics as stats
from math import isclose
from pathlib import Path
%matplotlib inline

#### Notes:
To import .mat files into python:  
import scipy.io   
mat = scipy.io.loadmat('file.mat')  

To unzip files: look at SEDS HW2 (not sure if we will use this)


#### CS2 data (in excel format*)  
Look at CS2 data first. From the website, CS2_'n' was given to the nth numbered CS2 cell (identifies unique cells from eachother). Each cell was cycled multiple times. Excel files are named using the testing dates.  **Were the same cells cycled on different dates or was the continuous cycling data split into multiple excel files?**   

*CS2_8 and CS2_21 are in .txt format because they were cycled using a different system.

Starting with CS2_33, in excel file format

In [2]:
#data1 = pd.read_excel("data/CS2_33/CS2_33_10_04_10.xlsx", sheet_name= "Channel_1-006")

#it looks like all of these have the sheet name channel_1-006, not just this specific excel file? 
#would be nice to create a function to iterate over all the files in a given file and import them, store them in variable. 

In [3]:
#data1['Cycle_Index'].max()

#this gives the number of cycles they ran

In [4]:
#we will need to clean the data so that dq/dv does not come back with weird data, ie we need to make sure the dq/dv isn't being 
# calculated between identical voltage values (because then dv = 0)
# (dq/dt)/(dv/dt)= dq/dv 
# (dq/dt) = current 


# calculate dv, find values close to 0, delete those rows, reindex, calculate dv again, 

### From Robert's Notebook:

In [5]:
### Point this bad boy at a directory full of Excel spreadsheets. 
### For each spreadsheet in the directory, it will spit the 2nd sheet's data into a pandas dataframe. 
### These dataframes go into a dictionary, with each key named 
### after the original name of the .xlsx spreadsheet.

rootdir = Path('data')
# Return a list of regular files only, not directories
file_list = [f for f in rootdir.glob('**/*') if f.is_file()]
### Source: https://stackoverflow.com/questions/19587118/iterating-through-directories-with-python/19587581

d = {}
count = 0
for file in file_list:
    count += 1
    name = os.path.split(file)[1].split('.')[0] 
    data = pd.read_excel(file,1)
    new_set = {name : data}
    d.update(new_set)
    print(count, name)
print('Done getting files')
### There are 23 files in the CS2 directory, so we should have 23 entries in the dictionary

1 CS2_33_10_04_10
2 CS2_33_10_05_10
3 CS2_33_10_15_10
4 CS2_33_10_26_10
5 CS2_33_11_01_10
6 CS2_33_11_10_10
7 CS2_33_11_19_10
8 CS2_33_11_24_10
9 CS2_33_12_08_10
10 CS2_33_12_16_10
11 CS2_33_12_23_10
12 CS2_33_1_10_11
13 CS2_33_1_18_11
14 CS2_33_1_24_11
15 CS2_33_1_28_11
16 CS2_33_2_2_11
17 CS2_33_8_17_10
18 CS2_33_8_18_10
19 CS2_33_8_19_10
20 CS2_33_8_30_10
21 CS2_33_9_17_10
22 CS2_33_9_27_10
23 CS2_33_9_7_10
Done getting files


In [26]:
data1 = d['CS2_33_10_04_10']

In [ ]:
data1['Cycle_Index'].max()

In [22]:
#iterate over keys of data to add a battery label and a column of None dV values
def add_label_dv_cols(dictionary):
    for keys in dictionary:
        data1 = dictionary[keys]
        # add a battery label corresponding to that battery's key. 
        data1['Battery_Label'] = keys
        #create column labeled 'dV'
        data1['dV'] = None
        data1['dQ/dV'] = None 

In [23]:
add_label_dv_cols(dictionary = d)

In [31]:
#separate out dataframes into cycles
def sep_cycles(dataframe):
    """This function separates out the cycles in the battery dataframe by grouping by the 'Cycle_Index' column, and putting them in a dictionary. """
    gb = dataframe.groupby(by = ['Cycle_Index'])
    cycle_dict = dict(iter(gb))
    return cycle_dict

In [55]:
test = sep_cycles(data1)
#test is a dictionary of cycles

In [51]:
writer = ExcelWriter('NLToutput.xlsx')
test[1].to_excel(writer)
writer.save()

In [56]:
len(test)

23

In [63]:
for i in range(1,len(test)+1):
    print(i)

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23


In [80]:
def save_sep_cycles_xlsx(cycle_dict, battname, path_to_folder):
    '''This saves the separated out cycles into different excel files, beginning with the battery name. Battname and path to folder must be strings.'''
    for i in range(1,len(cycle_dict)+1):
        writer = ExcelWriter(path_to_folder + battname + 'Cycle' + str(i) + '.xlsx')
        cycle_dict[i].to_excel(writer)
        writer.save()

In [79]:
save_sep_cycles_xlsx(test, 'NLTtest', 'C:/Users/Nicole/Desktop/DIRECTHW/chachies/ourproject/SeparatedCyclesXlsx/')

In [ ]:
## Actual cleaning 
    
    #calculate initial dV and initial dq/dv for each data point
    for i in range(1,len(data1)): 
        data1.loc[i, ('dV')] = data1.loc[i, ('Voltage(V)')] - data1.loc[i-1, ('Voltage(V)')]
    #calculate dq/dv based off of discharge capacity - might change this later so user can choose to use charge or discharge cap. 
    data1['dQ/dV-dis'] = data1['Discharge_Capacity(Ah)']/data1['dV']
    
    #drop rows if dv is about 0, this will clean up the data points around V = 4.2V (since they are holding it at 4.2V for a while)
       for i in range(1,len(data1)):
        if isclose(data1.loc[i, ('dV')], 0, abs_tol = 10**-3):
            data1 = data1.drop(index = i)
    #reset index
    data1 = data1.reset_index(drop = True)
    #recalculating dv after dropping rows
    for i in range(1,len(data1)): 
        data1.loc[i, ('dV')] = data1.loc[i, ('Voltage(V)')] - data1.loc[i-1, ('Voltage(V)')]
    #recalculate dq/dv  
    data1['dQ/dV-dis'] = data1['Discharge_Capacity(Ah)']/data1['dV']
    #will need to drope na row, there should only be one. 
    data1 = data1.dropna(subset=['dQ/dV-dis'], inplace= True)
    data1 = data1.reset_index(drop = True)
    #recalculate dq/dv after dropping n/a row
    data1['dQ/dV-dis'] = data1['Discharge_Capacity(Ah)']/data1['dV']
    
        
        
        #grab group, grab discharge/charge, smooth data, grab descriptors, put into a new data frame with battery type, descriptors.
        #this will be the dataframe we use for the ML 
        
        
    
   

In [ ]:
data1.head()

### Grab individual cycles to run above code on: 
#### Should label type of battery in column for each cycle somehow so we can relate individual cycles back to type of battery

In [ ]:
data1['Cycle_Index'].max()
#this gives number of cycles in a given set 

In [ ]:
gb = data1.groupby(by = ['Cycle_Index'])

In [ ]:
for name, group in gb: 
    print(name)
    #want to use this logic to put individual cycles into individual dataframes. How should we store these? 


In [ ]:
for name, group in gb:
    print(name)
    print(group)

In [ ]:
for i in range(1, data1['Cycle_Index'].max() + 1):
    print(i)
    

### Separate by charge/discharge 
#### We want to take each cycle as an individual, but we also want to separate into charge+discharge just to do gaussian fitting easier, but still take charge+discharge as one set. So after cycles are separated out to be treated as individuals, we will run above code to smooth, separate them into charge and discharge (could reverse those two steps) and then do gaussian fitting to find peak descriptors. 

New Column- To calculate dq/dv do discharge/charge capacity/ dv. calculate dv first. Ask robert whether to use discharge or charge capacity to calculate this. or do both and compare. Maybe ask user which capacity to use between discharge and charge - depends on system (i.e. Maccor) , which anode or cathode they are looking at etc. 

In [ ]:
data1['dV'] = None 

In [ ]:
data1.loc[2, ('Voltage(V)')]

In [ ]:
range(len(data1))

In [ ]:
for i in range(1,len(data1)): 
    data1.loc[i, ('dV')] = data1.loc[i, ('Voltage(V)')] - data1.loc[i-1, ('Voltage(V)')]


In [ ]:
data1['dQ/dV-dis'] = data1['Discharge_Capacity(Ah)']/data1['dV']

In [ ]:
#data1 = data1.replace([np.inf, -np.inf], np.nan).dropna(subset=['dQ/dV-dis'])

In [ ]:
#data1 = data1.reset_index(drop = True)

In [ ]:
data1['dQ/dV-dis'].mean()

In [ ]:
df1 = pd.DataFrame() 
for i in range(1,len(data1)):
    if data1.loc[i, ('Cycle_Index')] == 7:
        df1 = df1.append(data1.loc[i])
        
fig = plt.figure(figsize=(10, 10), facecolor='w', edgecolor='k')
myplot = plt.plot(df1['Voltage(V)'], df1['dQ/dV-dis'])

In [ ]:
for i in range(1,len(data1)):
    if isclose(data1.loc[i, ('dV')], 0, abs_tol = 10**-3):
        data1 = data1.drop(index = i)
#drop rows if dv is about 0, this will clean up the data points around V = 4.2V (since they are holding it at 4.2V for a while)
#have to recalculate the dv after this I think 

In [ ]:
len(data1)

In [ ]:
data1 = data1.reset_index(drop = True)

In [ ]:
#recalculating dv after dropping rows
for i in range(1,len(data1)): 
    data1.loc[i, ('dV')] = data1.loc[i, ('Voltage(V)')] - data1.loc[i-1, ('Voltage(V)')]


In [ ]:
#recalculate dq/dv  
data1['dQ/dV-dis'] = data1['Discharge_Capacity(Ah)']/data1['dV']

In [ ]:
df1 = pd.DataFrame() 
for i in range(1,len(data1)):
    if data1.loc[i, ('Cycle_Index')] == 7:
        df1 = df1.append(data1.loc[i])
        
fig = plt.figure(figsize=(10, 10), facecolor='w', edgecolor='k')
myplot = plt.plot(df1['Voltage(V)'], df1['dQ/dV-dis'])

In [ ]:
data1['dQ/dV-char'] = data1['Charge_Capacity(Ah)']/data1['dV']

In [ ]:
df1 = pd.DataFrame() 
for i in range(1,len(data1)):
    if data1.loc[i, ('Cycle_Index')] == 7:
        df1 = df1.append(data1.loc[i])
        
fig = plt.figure(figsize=(10, 10), facecolor='w', edgecolor='k')
myplot = plt.plot(df1['Voltage(V)'], df1['dQ/dV-char'])
# This is to see if using the charging capacity looks any different than using the discharging capacity. Look very similar. 

In [ ]:
data1['dQ/dV-dis'].max()

In [ ]:
data1['dQ/dV-dis'].min()

In [ ]:
data1['dQ/dV-char'].max()

In [ ]:
data1['dQ/dV-char'].min()

In [ ]:
data1.shape

Drop Rows With N/A in dq/dv-discharge cap row. First replace inf values for N/a values so can use dropna

In [ ]:
#will need to drope na row, there should only be one. 
data1.dropna(subset=['dQ/dV-dis'], inplace= True)

In [ ]:
data1.shape

In [ ]:
data1 = data1.reset_index(drop = True)

In [ ]:
#recalculate dq/dv after dropping n/a row
data1['dQ/dV-dis'] = data1['Discharge_Capacity(Ah)']/data1['dV']

Reindex and calculate dV again 

In [ ]:
df1 = pd.DataFrame() 
for i in range(1,len(data1)):
    if data1.loc[i, ('Cycle_Index')] == 7:
        df1 = df1.append(data1.loc[i])
        
fig = plt.figure(figsize=(10, 10), facecolor='w', edgecolor='k')
myplot = plt.plot(df1['Voltage(V)'], df1['dQ/dV-dis'])
 

### Filter Sav Golay

In [ ]:
data1['dV'].describe()

In [ ]:
unfilt = pd.concat([data1['dQ/dV-dis']])

In [ ]:
unfiltar = unfilt.values
#converts into an array 

In [ ]:
data1['filtered dq/dv'] = scipy.signal.savgol_filter(unfiltar, 21, 3)

In [ ]:
df1 = pd.DataFrame() 
for i in range(1,len(data1)):
    if data1.loc[i, ('Cycle_Index')] == 6:
        df1 = df1.append(data1.loc[i])
        
fig = plt.figure(figsize=(10, 10), facecolor='w', edgecolor='k')
myplot = plt.plot(df1['Voltage(V)'], df1['filtered dq/dv'])